In [ ]:
import pandas as pd
import os
import numpy as np
import random
import datetime
import warnings
warnings.filterwarnings('ignore')

# functions

In [ ]:
def add_barcode_information(table, barcode_information, level='barcode'):
    num_elements = len(table)
    if level=='barcode':
        table['barcode'] = table['barcode'].astype(np.int64)
        barcode_information['barcode'] = barcode_information['barcode'].astype(np.int64)
        table['producer_name'] = table['barcode'].map(dict(zip(barcode_information['barcode'], barcode_information['producer'])))
        table['brand_name'] = table['barcode'].map(dict(zip(barcode_information['barcode'], barcode_information['brand'])))
        table['name_val'] = table['barcode'].map(dict(zip(barcode_information['barcode'], barcode_information['product_name'])))
       
    elif level=='brand_name':
        table['brand_name'] = table['brand_name'].astype(str)
        barcode_information['brand'] = barcode_information['brand'].astype(str)
        barcode_information = barcode_information[['brand', 'producer']]
        barcode_information.drop_duplicates(inplace=True)
        table['producer_name'] = table['brand_name'].map(dict(zip(barcode_information['brand'], barcode_information['producer'])))
    if num_elements == len(table):
        return table
    else: return print('ERROR->add_barcode_information')

In [ ]:
def add_size_type(table, barcode_information):
    num_elements = len(table)
    table['size_of_package(litres)'] = table['barcode'].map(dict(zip(barcode_information['barcode'], barcode_information['size_of_package'])))
#     table['type_of_package'] = table['barcode'].map(dict(zip(barcode_information['barcode'], barcode_information['type_of_package'])))
    if num_elements == len(table):
        return table
    else: return print('ERROR->add_size_type')

In [ ]:
def get_right_order(table):
    lll = ['producer_name','brand_name','name_val','barcode', 'size_of_package(litres)', 'type_of_package',
           'value(kzt)','value(%)','items','items(%)', 'volume(litre)', 'volume(%)',
           'price', 'price_per_litre', 'price_segmentation', 'coverage(%)','weighted_distribution(%)',
           'region','period','category', 'sub_category']
    col = list(table.columns)
    ttt = []
    for i in lll:
        if i in col: ttt.append(i)
    table = table[ttt]
    if len(col) == len(table.columns):
        return table
    else: 
        return print('ERROR->get_right_order')

In [ ]:
def get_main_factors(df,coef, region, barcode_information):
    length = len(df)
    num_bar = df['barcode'].nunique()
    df['product_quantity'] = df['product_quantity'].astype(int)
    df['total_sale_amount'] = df['total_sale_amount'].astype(int)
    g_bar = df.groupby(['barcode']).sum()[['product_quantity','total_sale_amount']].reset_index()
    g_bar.columns = ['barcode', 'items', 'value(kzt)']
    g_bar = g_bar[['barcode', 'value(kzt)', 'items']]
    g_bar = add_size_type(g_bar, barcode_information)
    g_bar['volume(litre)'] = g_bar['items'] * g_bar['size_of_package(litres)']
    
    g_bar['value(kzt)'] = g_bar['value(kzt)'] / 1000000
    g_bar['items'] = g_bar['items'] / 1000
    g_bar['volume(litre)'] = g_bar['volume(litre)'] / 1000
    g_bar['value(kzt)'] = g_bar['value(kzt)'] * coef
    g_bar['items'] = g_bar['items'] * coef
    g_bar['volume(litre)'] = g_bar['volume(litre)'] * coef
    
    sss = g_bar['value(kzt)'].sum()
    g_bar['value(%)'] = g_bar['value(kzt)'] / sss * 100
    sss = g_bar['items'].sum()
    g_bar['items(%)'] = g_bar['items'] / sss * 100    
    sss = g_bar['volume(litre)'].sum()
    g_bar['volume(%)'] = g_bar['volume(litre)'] / sss * 100
    
    g_bar['price'] = g_bar['value(kzt)'] / g_bar['items'] * 1000
    g_bar['price'] = g_bar['price'].round()
    g_bar['price_per_litre'] = g_bar['price'] / g_bar['size_of_package(litres)']
    g_bar['price_per_litre'] = g_bar['price_per_litre'].round(3)
    g_bar['region'] = region
    if len(g_bar) == num_bar:
        return g_bar
    else: return print('ERROR->get_sales_items_price')

In [ ]:
import decimal
def float_range(start, stop, step):
    while start < stop:
        yield float(start)
        start+=decimal.Decimal(step)

def change_coverage(value, lll):
    for i in sorted(lll):
        if float(i) >= float(value):
            return i

In [ ]:
def get_coverage(df, grouped_data, total_stores, region, level='barcode'):
    num_elements = grouped_data[level].nunique()
    df['product_quantity'] = df['product_quantity'].astype(int)
    df['total_sale_amount'] = df['total_sale_amount'].astype(int)
    
    coverage_table = df.groupby([level]).nunique()['store_id'].reset_index()
    coverage_table['coverage(%)'] = coverage_table['store_id'] / total_stores*100
    coverage_table = coverage_table[[level, 'coverage(%)']]
    coverage_table[level] = coverage_table[level].astype(str)
    grouped_data[level] = grouped_data[level].astype(str)
    grouped_data = grouped_data.merge(coverage_table, on=[level], how='left')
    grouped_data.reset_index(drop=True, inplace=True)
    
    if region == 'Kyzylorda':
        lll_for_cov = list(float_range(0,150.1,str(1/15*100)))
        for i in range(len(grouped_data)):
            grouped_data.at[i, 'coverage(%)'] = change_coverage(grouped_data.iloc[i]['coverage(%)'], lll_for_cov)
    elif region == 'Kostanay':
        lll_for_cov = list(float_range(0,150.1,str(1/11*100)))
        for i in range(len(grouped_data)):
            grouped_data.at[i, 'coverage(%)'] = change_coverage(grouped_data.iloc[i]['coverage(%)'], lll_for_cov)
    grouped_data['coverage(%)'] = grouped_data['coverage(%)'].round(3)
    if num_elements == len(grouped_data):
        return grouped_data
    else: return print('ERROR->get_coverage')

In [ ]:
def get_weighted_distribution(df, grouped_data, level='barcode'):
    num_elements = grouped_data[level].nunique()
    df['product_quantity'] = df['product_quantity'].astype(int)
    df['total_sale_amount'] = df['total_sale_amount'].astype(int)
    g_store = df.groupby('store_id').sum()['product_quantity'].reset_index()
    sss = g_store['product_quantity'].sum()
    g_store['weighted_store'] = g_store['product_quantity'] / sss*100
    temp_temp = df[['store_id', level]]
    temp_temp.drop_duplicates(inplace=True)
    temp_temp['weighted_store'] = temp_temp['store_id'].map(dict(zip(g_store['store_id'], g_store['weighted_store'])))
    temp_temp.columns = ['store_id',level,'weighted_distribution(%)'] 
    wd = temp_temp.groupby(level).sum()['weighted_distribution(%)'].reset_index()
    grouped_data[level] = grouped_data[level].astype(str)
    wd[level] = wd[level].astype(str)
    grouped_data = grouped_data.merge(wd, on=[level], how='left')
    grouped_data['weighted_distribution(%)'] = grouped_data['weighted_distribution(%)'].round(3)
    
    if num_elements == len(grouped_data):
        return grouped_data
    else: return print('ERROR->get_weighted_distribution')

In [ ]:
def group_by_brand_company(df, grouped_data, region, total_stores, level='brand_name'):
    if level=='brand_name':
        num_elements = grouped_data[level].nunique()

        g_level = grouped_data.groupby(level).sum()[['value(kzt)', 'items','volume(litre)']].reset_index()
        sss = g_level['value(kzt)'].sum()
        g_level['value(%)'] = g_level['value(kzt)'] / sss * 100
        sss = g_level['items'].sum()
        g_level['items(%)'] = g_level['items'] / sss * 100

        sss = g_level['volume(litre)'].sum()
        g_level['volume(%)'] = g_level['volume(litre)'] / sss * 100

        g_level['price'] = g_level['value(kzt)'] / g_level['items'] * 1000

        g_level['price'] = g_level['price'].round()
        g_level['region'] = region
        g_level = get_coverage(df, g_level, total_stores, region, level)
        g_level = get_weighted_distribution(df, g_level, level)
        return g_level
    elif level=='producer_name':
        num_elements = grouped_data[level].nunique()

        g_level = grouped_data.groupby(level).sum()[['value(kzt)', 'items','volume(litre)']].reset_index()
        sss = g_level['value(kzt)'].sum()
        g_level['value(%)'] = g_level['value(kzt)'] / sss * 100
        sss = g_level['items'].sum()
        g_level['items(%)'] = g_level['items'] / sss * 100

        sss = g_level['volume(litre)'].sum()
        g_level['volume(%)'] = g_level['volume(litre)'] / sss * 100

        g_level['price'] = g_level['value(kzt)'] / g_level['items'] * 1000

        g_level['price'] = g_level['price'].round()
        g_level['region'] = region
        g_level = get_coverage(df, g_level, total_stores, region, level)
        g_level = get_weighted_distribution(df, g_level, level)
        return g_level
        


In [ ]:
def collect_cities_barcode(df,barcode_information,level='barcode'):
    if level=='barcode':
        temp_almaty=df[df.region.str.startswith('Almaty')]
        temp_almaty['region']='Almaty'
        temp_almaty1=temp_almaty.groupby(by=['barcode','region']).max()[['numeric distribution(%)','weighted distribution(%)']].reset_index()
        temp_almaty=temp_almaty.groupby(by=['barcode','region']).sum()[['value(kzt)','items','volume(litre)']].reset_index()
        
        temp_almaty=temp_almaty.merge(temp_almaty1,on=['barcode','region'],how='left')
        
        temp_almaty['avg. selling price']=(temp_almaty['value(kzt)']/temp_almaty['items']*1000).round()
        temp_almaty = add_size_type(temp_almaty, barcode_information)        
        temp_almaty['avg. price per litre']= (temp_almaty['avg. selling price'] / temp_almaty['size_of_package(litres)']).round(3)
        
        temp_almaty = add_barcode_information(temp_almaty, barcode_information_table, level='barcode') 
        temp_almaty.rename(columns={'producer_name':'producer','brand_name':'brand','name_val':'product name'},inplace=True)
        temp_almaty['category']=temp_almaty['barcode'].map(dict(zip(barcode_information_table['barcode'],barcode_information_table['category'])))
        temp_almaty['sub_category']=temp_almaty['barcode'].map(dict(zip(barcode_information_table['barcode'],barcode_information_table['sub_category'])))
        temp_almaty['CDE'] = temp_almaty['weighted distribution(%)'] / temp_almaty['numeric distribution(%)']
        temp_almaty['CDE'] = temp_almaty['CDE'].round(4)



        temp_astana=df[df.region.str.startswith('Nur-Sultan')]
        temp_astana['region']='Nur-Sultan'
        temp_astana1=temp_astana.groupby(by=['barcode','region']).max()[['numeric distribution(%)','weighted distribution(%)']].reset_index()
        temp_astana=temp_astana.groupby(by=['barcode','region']).sum()[['value(kzt)','items','volume(litre)']].reset_index()
        
        temp_astana=temp_astana.merge(temp_astana1,on=['barcode','region'],how='left')
        
        temp_astana['avg. selling price']=(temp_astana['value(kzt)']/temp_astana['items']*1000).round()
        temp_astana = add_size_type(temp_astana, barcode_information)        
        temp_astana['avg. price per litre']= (temp_astana['avg. selling price'] / temp_astana['size_of_package(litres)']).round(3)
        
        
        temp_astana = add_barcode_information(temp_astana, barcode_information_table, level='barcode') 
        temp_astana.rename(columns={'producer_name':'producer','brand_name':'brand','name_val':'product name'},inplace=True)
        temp_astana['category']=temp_astana['barcode'].map(dict(zip(barcode_information_table['barcode'],barcode_information_table['category'])))
        temp_astana['sub_category']=temp_astana['barcode'].map(dict(zip(barcode_information_table['barcode'],barcode_information_table['sub_category'])))
        
        
        
        temp_astana['CDE'] = temp_astana['weighted distribution(%)'] / temp_astana['numeric distribution(%)']
        temp_astana['CDE'] = temp_astana['CDE'].round(4)

        temp_kz=df.copy()
        temp_kz['region']='Kazakhstan'
        temp_kz_add=temp_kz.groupby(by=['barcode','region']).max()[['numeric distribution(%)','weighted distribution(%)']].reset_index()
        temp_kz=temp_kz.groupby(by=['barcode','region']).sum()[['value(kzt)','items','volume(litre)']].reset_index()
        temp_kz=temp_kz.merge(temp_kz_add,on=['barcode','region'],how='left')
        temp_kz['avg. selling price']=(temp_kz['value(kzt)']/temp_kz['items']*1000).round()
        
        temp_kz = add_size_type(temp_kz, barcode_information)        
        temp_kz['avg. price per litre']= (temp_kz['avg. selling price'] / temp_kz['size_of_package(litres)']).round(3)
        
        
        temp_kz = add_barcode_information(temp_kz, barcode_information_table, level='barcode') 
        temp_kz.rename(columns={'producer_name':'producer','brand_name':'brand','name_val':'product name'},inplace=True)
        temp_kz['category']=temp_kz['barcode'].map(dict(zip(barcode_information_table['barcode'],barcode_information_table['category'])))
        temp_kz['sub_category']=temp_kz['barcode'].map(dict(zip(barcode_information_table['barcode'],barcode_information_table['sub_category'])))
        
        
        
        temp_kz['CDE'] = temp_kz['weighted distribution(%)'] / temp_kz['numeric distribution(%)']
        temp_kz['CDE'] = temp_kz['CDE'].round(4)

        df=df.append(temp_almaty,ignore_index=True)
        df=df.append(temp_astana,ignore_index=True)
        df=df.append(temp_kz,ignore_index=True)        
    return df

In [ ]:
# for collecting
def add_barcode_information_brand(table, barcode_information, level='brand'):
    if level=='brand':
        table['brand'] = table['brand'].astype(str)
        barcode_information['brand'] = barcode_information['brand'].astype(str)
        barcode_information = barcode_information[['brand', 'producer']]
        barcode_information.drop_duplicates(inplace=True)
        table['producer'] = table['brand'].map(dict(zip(barcode_information['brand'], barcode_information['producer'])))
        return table    

In [ ]:
def collect_cities_brand(df,barcode_information,level='brand'):
    if level=='brand':
        temp_almaty=df[df.region.str.startswith('Almaty')]
        temp_almaty['region']='Almaty'
        temp_almaty1=temp_almaty.groupby(by=['brand','region','category','sub_category']).max()[['numeric distribution(%)','weighted distribution(%)']].reset_index()
        temp_almaty=temp_almaty.groupby(by=['brand','region','category','sub_category']).sum()[['value(kzt)','items','volume(litre)']].reset_index()        
        temp_almaty=temp_almaty.merge(temp_almaty1,on=['brand','region','category','sub_category'],how='left')        
        temp_almaty['avg. selling price']=(temp_almaty['value(kzt)']/temp_almaty['items']*1000).round()
        temp_almaty = add_barcode_information_brand(temp_almaty, barcode_information_table, level='brand') 
#         temp_almaty['category']=temp_almaty['brand'].map(dict(zip(barcode_information_table['brand'],barcode_information_table['category'])))
#         temp_almaty['sub_category']=temp_almaty['brand'].map(dict(zip(barcode_information_table['brand'],barcode_information_table['sub_category'])))
        temp_almaty['CDE'] = temp_almaty['weighted distribution(%)'] / temp_almaty['numeric distribution(%)']
        temp_almaty['CDE'] = temp_almaty['CDE'].round(4)
        df=df.append(temp_almaty,ignore_index=True)
        
        temp_astana=df[df.region.str.startswith('Nur-Sultan')]
        temp_astana['region']='Nur-Sultan'
        temp_astana1=temp_astana.groupby(by=['brand','region','category','sub_category']).max()[['numeric distribution(%)','weighted distribution(%)']].reset_index()
        temp_astana=temp_astana.groupby(by=['brand','region','category','sub_category']).sum()[['value(kzt)','items','volume(litre)']].reset_index()        
        temp_astana=temp_astana.merge(temp_astana1,on=['brand','region','category','sub_category'],how='left')        
        temp_astana['avg. selling price']=(temp_astana['value(kzt)']/temp_astana['items']*1000).round()
        temp_astana = add_barcode_information_brand(temp_astana, barcode_information_table, level='brand') 
#         temp_astana['category']=temp_astana['brand'].map(dict(zip(barcode_information_table['brand'],barcode_information_table['category'])))
#         temp_astana['sub_category']=temp_astana['brand'].map(dict(zip(barcode_information_table['brand'],barcode_information_table['sub_category'])))
        temp_astana['CDE'] = temp_astana['weighted distribution(%)'] / temp_astana['numeric distribution(%)']
        temp_astana['CDE'] = temp_astana['CDE'].round(4)        
        df=df.append(temp_astana,ignore_index=True)
        
        temp_kz=df.copy()
        temp_kz['region']='Kazakhstan'
        temp_kz_add=temp_kz.groupby(by=['brand','region','category','sub_category']).max()[['numeric distribution(%)','weighted distribution(%)']].reset_index()
        temp_kz=temp_kz.groupby(by=['brand','region','category','sub_category']).sum()[['value(kzt)','items','volume(litre)']].reset_index()
        temp_kz['avg. selling price']=(temp_kz['value(kzt)']/temp_kz['items']*1000).round()
        temp_kz = add_barcode_information_brand(temp_kz, barcode_information_table, level='brand') 
#         temp_kz['category']=temp_kz['brand'].map(dict(zip(barcode_information_table['brand'],barcode_information_table['category'])))
#         temp_kz['sub_category']=temp_kz['brand'].map(dict(zip(barcode_information_table['brand'],barcode_information_table['sub_category'])))
        temp_kz=temp_kz.merge(temp_kz_add,on=['brand','region','category','sub_category'],how='left')        
        temp_kz['CDE'] = temp_kz['weighted distribution(%)'] / temp_kz['numeric distribution(%)']
        temp_kz['CDE'] = temp_kz['CDE'].round(4)       
        df=df.append(temp_kz,ignore_index=True)
        
    elif level=='producer':
        temp_almaty=df[df.region.str.startswith('Almaty')]
        temp_almaty['region']='Almaty'
        temp_almaty1=temp_almaty.groupby(by=['producer','region','category','sub_category']).max()[['numeric distribution(%)','weighted distribution(%)']].reset_index()
        temp_almaty=temp_almaty.groupby(by=['producer','region','category','sub_category']).sum()[['value(kzt)','items','volume(litre)']].reset_index()
        temp_almaty=temp_almaty.merge(temp_almaty1,on=['producer','region','category','sub_category'],how='left')        
        temp_almaty['avg. selling price']=(temp_almaty['value(kzt)']/temp_almaty['items']*1000).round()        
        temp_almaty['CDE'] = temp_almaty['weighted distribution(%)'] / temp_almaty['numeric distribution(%)']
        temp_almaty['CDE'] = temp_almaty['CDE'].round(4)
        df=df.append(temp_almaty,ignore_index=True)
        
        temp_astana=df[df.region.str.startswith('Nur-Sultan')]
        temp_astana['region']='Nur-Sultan'
        temp_astana1=temp_astana.groupby(by=['producer','region','category','sub_category']).max()[['numeric distribution(%)','weighted distribution(%)']].reset_index()
        temp_astana=temp_astana.groupby(by=['producer','region','category','sub_category']).sum()[['value(kzt)','items','volume(litre)']].reset_index()
        temp_astana=temp_astana.merge(temp_astana1,on=['producer','region','category','sub_category'],how='left')        
        temp_astana['avg. selling price']=(temp_astana['value(kzt)']/temp_astana['items']*1000).round()        
        temp_astana['CDE'] = temp_astana['weighted distribution(%)'] / temp_astana['numeric distribution(%)']
        temp_astana['CDE'] = temp_astana['CDE'].round(4)
        df=df.append(temp_astana,ignore_index=True)
        
        temp_kz=df.copy()
        temp_kz['region']='Kazakhstan'
        temp_kz1=temp_kz.groupby(by=['producer','region','category','sub_category']).max()[['numeric distribution(%)','weighted distribution(%)']].reset_index()
        temp_kz=temp_kz.groupby(by=['producer','region','category','sub_category']).sum()[['value(kzt)','items','volume(litre)']].reset_index()
        temp_kz=temp_kz.merge(temp_kz1,on=['producer','region','category','sub_category'],how='left')        
        temp_kz['avg. selling price']=(temp_kz['value(kzt)']/temp_kz['items']*1000).round()        
        temp_kz['CDE'] = temp_kz['weighted distribution(%)'] / temp_kz['numeric distribution(%)']
        temp_kz['CDE'] = temp_kz['CDE'].round(4)
        df=df.append(temp_kz,ignore_index=True)        
    return df

In [ ]:
d_month = {'december':'2021-12'}
for month in list(d_month.keys()):
    print(month)
    data = pd.read_csv(r'D:\Bekbol working\PRK\\'+d_month[month]+'.csv')
    barcode_information_table=pd.read_excel(r'C:\Users\svnduw\Desktop\Bekbol\Chingis\PRK\barcode_information.xlsx')
    barcode_information_table.barcode=barcode_information_table.barcode.astype(np.int64)
    stores_data=pd.read_csv(r'C:\Users\svnduw\Desktop\Bekbol\Chingis\general_data\store_id.csv')
    stores_data=stores_data[(stores_data[f'{month}']==True) & (stores_data['region']!='Bishkek')]
    stores_data = stores_data[['store_id', 'region']]
    stores_data.columns = ['store_id', 'region_new']
    barcode_information_table = barcode_information_table[['category', 'sub_category', 'producer', 'brand', 'barcode', 'product_name',
            'size_of_package', 'type_of_package']]
    barcode_information_table.columns = ['category', 'sub_category', 'producer', 'brand', 'barcode',
           'product_name', 'size_of_package', 'type_of_package']
    data = add_barcode_information(data, barcode_information_table, level='barcode')
    if month == 'january':
        kyzyl_stores = [2147, 2670, 5114, 6711, 5471, 5612, 5108]
        kostanay_stores = [5719, 6747, 5229, 6061, 5639, 4234, 6957, 6490, 5780, 6957, 6481]
    elif month == 'february':
        kyzyl_stores = [2147, 2670, 5114, 6711, 2611, 5471, 5612, 5108]
        kostanay_stores = [5719, 6747, 5229, 6061, 5639, 4234, 6957, 6490, 5780, 6957, 6481]
    elif month == 'march':
        kyzyl_stores = [2147, 2670, 5114, 6711, 2611, 5471, 5612, 5108]
        kostanay_stores = [4234,5229,5322,5639,5719,5780,6061,6481,6490,6747,6957]
    elif month == 'april':
        kyzyl_stores = [2147, 2670, 5114, 6711, 2611, 5471, 5612, 5108]
        kostanay_stores = [4234,5229,5322,5639,5719,6061,6481,6490,6747,6957, 8552]
    if month == 'may':
        kyzyl_stores = [2147, 2670, 5114, 6711, 2611, 9147, 5471, 5612]#may
        kostanay_stores = [4234,5229,5322,5639,5719,6061,6481,6490,6957, 8552, 7114]
    if month == 'june':
        kyzyl_stores = [2147, 2670, 5114, 6711, 2611, 9147, 8156, 5471, 4088]#june
        kostanay_stores = [5322, 6957, 5229, 9863, 5719, 5639, 8552, 4234, 6481, 6061,6490]
    if month == 'july':
        kyzyl_stores = [2147, 2670, 5114, 6711, 2611, 8874, 8156, 5471, 10665, 5108]#july
        kostanay_stores = [5322, 6957, 5229, 9863, 5719, 5639, 8552, 4234, 6481, 6061,6490]
    if month == 'august':
        kyzyl_stores = [2147, 2670, 5114, 6711, 2611, 6214, 10777, 8874, 8156, 5471, 4088, 5108]#august
        kostanay_stores = [8552, 9863, 6061, 5229, 5719, 6481, 5322, 6490, 4234]
    if month == 'september':
        kyzyl_stores = [2147, 2670, 5114, 6711, 2611, 6214, 10777, 8874, 8156, 5471, 10665, 4088]#september
        kostanay_stores = [8552, 9863, 6061, 5229, 5719, 6481, 5322, 6490, 4234]
    if month == 'october':
        kyzyl_stores = [2147, 2670, 5114, 6711, 2611, 6214, 10777, 8874, 8156, 5471, 10665, 4088, 5108]#september
        kostanay_stores = [6481, 6061, 6490, 4234, 8552, 5229, 9863, 5322, 11506]
    if month == 'november':
        kyzyl_stores = [2147, 2670, 5114, 6711, 2611, 10777, 10671, 8874, 8156, 5471, 11676, 10665, 4088, 5108]#november
        kostanay_stores = [8552,  5719,  6481,  5229,  6490,  5322,  9863, 11506]
    if month == 'december':
        kyzyl_stores = [2147, 2670, 5114, 6711, 13911, 1832, 2611, 6214, 10777, 10671, 8874, 8156, 5471, 11676, 10665, 4088, 5108]#december
        kostanay_stores = [6481,  8552,  5719,  5229, 11506,  6490,  5322]
    universe_data = pd.read_excel(r'C:\Users\svnduw\Desktop\Bekbol\Chingis\general_data\universe_data.xlsx', sheet_name='others')
    universe_data1 = pd.read_excel(r'C:\Users\svnduw\Desktop\Bekbol\Chingis\general_data\universe_data.xlsx', sheet_name='Almaty')
    universe_data2 = pd.read_excel(r'C:\Users\svnduw\Desktop\Bekbol\Chingis\general_data\universe_data.xlsx', sheet_name='Astana')
    universe_data = pd.concat([universe_data, universe_data1, universe_data2])
    if month in ['january', 'february', 'march']:
        universe_data = universe_data[['region','january']]
    elif month in ['april', 'may', 'june']:
        universe_data = universe_data[['region','february']]
    elif month in ['july', 'august', 'september']:
        universe_data = universe_data[['region','july']]
    elif month in ['october', 'november', 'december']:
        universe_data = universe_data[['region','november']]
    universe_data.columns = ['region', 'month']

In [ ]:
barcode_information_table.barcode=barcode_information_table.barcode.astype(np.int64)
data.barcode=data.barcode.astype(np.int64)
category='alcohol'
regions = ['Almaty - 6','Almaty - 22','Almaty - 18','Almaty - 13','Almaty - 19', 'Almaty - 27', 'Almaty - 3', 'Almaty - 7','Almaty - 24','Almaty - 10','Almaty - 21','Almaty - 26','Almaty - 16', 'Almaty - 28', 'Almaty - 15', 
            'Nur-Sultan-7','Almaty - 2', 'Almaty - 23', 'Nur-Sultan-11', 'Nur-Sultan-9','Almaty - 14', 'Almaty - 8', 'Taldykorgan', 'Almaty - 25','Atyrau', 'Almaty - 1', 'Nur-Sultan-6', 'Almaty - 5', 'Almaty - 9','Almaty - 12', 'Petropavlovsk', 'Nur-Sultan-16', 
            'Nur-Sultan-5','Almaty - 11', 'Nur-Sultan-17', 'Nur-Sultan-3', 'Nur-Sultan-10','Nur-Sultan-12', 'Nur-Sultan-13', 'Oral', 'Nur-Sultan-1','Nur-Sultan-15', 'Nur-Sultan-8', 'Aktau', 'Nur-Sultan-14','Oskemen', 'Almaty - 17', 'Taraz', 'Shymkent', 
            'Almaty - 20', 'Almaty - 4', 'Aktobe', 'Karagandy', 'Nur-Sultan-4', 'Turkestan','Semey', 'Pavlodar', 'Kokshetau', 'Kyzylorda', 'Kostanay', 'Nur-Sultan-2']
by_barcode = pd.DataFrame(columns = ['producer_name', 'brand_name', 'name_val', 'barcode', 'size_of_package(litres)','value(kzt)', 'value(%)','items',
                                     'items(%)','volume(litre)', 'volume(%)', 'price', 'price_per_litre','coverage(%)','weighted_distribution(%)', 'region','category', 'sub_category'])
by_brand = pd.DataFrame(columns=['producer_name', 'brand_name', 'value(kzt)', 'value(%)', 'items','items(%)','volume(litre)', 'volume(%)',
                                 'price', 'coverage(%)','weighted_distribution(%)', 'region','category', 'sub_category'])
by_company = pd.DataFrame(columns=['producer_name', 'value(kzt)', 'value(%)', 'items', 'items(%)','volume(litre)', 'volume(%)',
                                   'price', 'coverage(%)','weighted_distribution(%)', 'region','category', 'sub_category'])
sub_categories=list(barcode_information_table['sub_category'].unique())
for sub_cat in sub_categories:
    
    barcodes = barcode_information_table[barcode_information_table['sub_category'].isin([str(sub_cat)])]['barcode'].tolist()
    for region in regions:
        stores = list(stores_data[stores_data['region_new']==region]['store_id'].unique())
        if region == 'Kyzylorda': 
            stores = kyzyl_stores
        elif region == 'Kostanay': 
            stores = kostanay_stores            
        universe = int(universe_data[universe_data['region']==region]['month'])
        total_stores = int(data[data['store_id'].isin(stores)]['store_id'].nunique())
        coefficient = universe/total_stores
        temp = data[data['barcode'].isin(barcodes)]
        temp.barcode=temp.barcode.astype(np.int64)
        temp = temp[temp['store_id'].isin(stores)]        
        temp = add_barcode_information(temp, barcode_information_table, level='barcode')        
        g_table = get_main_factors(temp,coefficient, region, barcode_information_table)
        
        g_table = get_coverage(temp, g_table, total_stores, region, level='barcode')
        g_table = get_weighted_distribution(temp, g_table, level='barcode')
        g_table.barcode=g_table.barcode.astype(np.int64)
        g_table['region'] = region
        g_table['category'] = category
        g_table['sub_category'] = sub_cat
        g_table = add_barcode_information(g_table, barcode_information_table, level='barcode')       
        g_table = get_right_order(g_table)
        by_barcode=pd.concat([by_barcode,g_table],ignore_index=True)
        g_brand = group_by_brand_company(temp, g_table, region, total_stores, 'brand_name')
        g_brand = add_barcode_information(g_brand, barcode_information_table, 'brand_name')
        g_brand['region'] = region
        g_brand['category'] = category
        g_brand['sub_category'] = sub_cat
        g_brand = get_right_order(g_brand)
        by_brand = pd.concat([by_brand, g_brand], ignore_index=True)
        g_company = group_by_brand_company(temp, g_table, region, total_stores, 'producer_name')
        g_company['region'] = region
        g_company['category'] = category
        g_company['sub_category'] = sub_cat
        g_company = get_right_order(g_company)
        by_company = pd.concat([by_company, g_company], ignore_index=True)  
        
by_barcode.columns =  ['producer', 'brand', 'product name', 'barcode','size_of_package(litres)', 'value(kzt)', 'value(%)',
                       'items', 'items(%)', 'volume(litre)', 'volume(%)','avg. selling price','avg. price per litre',
                       'numeric distribution(%)', 'weighted distribution(%)', 'region', 'category', 'sub_category']
by_brand.columns = ['producer', 'brand', 'value(kzt)', 'value(%)', 'items', 'items(%)','volume(litre)', 'volume(%)',
                    'avg. selling price','numeric distribution(%)', 'weighted distribution(%)', 'region', 'category', 'sub_category']


by_company.columns = ['producer', 'value(kzt)', 'value(%)', 'items', 'items(%)','volume(litre)', 'volume(%)',
                      'avg. selling price','numeric distribution(%)', 'weighted distribution(%)', 'region', 'category', 'sub_category']

by_barcode['CDE'] = by_barcode['weighted distribution(%)'] / by_barcode['numeric distribution(%)']
by_brand['CDE'] = by_brand['weighted distribution(%)'] / by_brand['numeric distribution(%)']
by_company['CDE'] = by_company['weighted distribution(%)'] / by_company['numeric distribution(%)']

by_barcode['CDE'] = by_barcode['CDE'].round(4)
by_brand['CDE'] = by_brand['CDE'].round(4)
by_company['CDE'] = by_company['CDE'].round(4)

by_barcode = by_barcode[['producer', 'brand', 'product name', 'barcode','size_of_package(litres)', 'value(kzt)', 'value(%)',
                       'items', 'items(%)', 'volume(litre)', 'volume(%)','avg. selling price','avg. price per litre',
                       'numeric distribution(%)', 'weighted distribution(%)','CDE','region', 'category', 'sub_category']]
by_brand = by_brand[['producer', 'brand', 'value(kzt)', 'value(%)', 'items', 'items(%)','volume(litre)', 'volume(%)',
                    'avg. selling price','numeric distribution(%)', 'weighted distribution(%)','CDE','region', 'category', 'sub_category']]
by_company = by_company[['producer', 'value(kzt)', 'value(%)', 'items', 'items(%)','volume(litre)', 'volume(%)',
                      'avg. selling price','numeric distribution(%)', 'weighted distribution(%)','CDE', 'region', 'category', 'sub_category']]


by_barcode=collect_cities_barcode(by_barcode,barcode_information_table,level='barcode')
temp2=by_barcode.groupby(by=['region','category','sub_category']).sum()[['value(kzt)','items','volume(litre)']].reset_index()
temp2.rename(columns={'value(kzt)':'total_value','items':'total_items','volume(litre)':'total_volume'},inplace=True)
by_barcode=by_barcode.merge(temp2,on=['region','category','sub_category'],how='left')
by_barcode['value(%)']=by_barcode['value(kzt)']/by_barcode['total_value']*100
by_barcode['items(%)']=by_barcode['items']/by_barcode['total_items']*100
by_barcode['volume(%)']=by_barcode['volume(litre)']/by_barcode['total_volume']*100
by_barcode.drop(['total_value','total_items','total_volume'],axis=1,inplace=True)
by_barcode.sort_values(['region', 'producer','brand'], inplace=True)

by_brand=collect_cities_brand(by_brand,barcode_information_table,level='brand')
by_brand.dropna(subset=['numeric distribution(%)', 'weighted distribution(%)','CDE'],inplace=True)
temp3=by_brand.groupby(by=['region','category','sub_category']).sum()[['value(kzt)','items','volume(litre)']].reset_index()
temp3.rename(columns={'value(kzt)':'total_value','items':'total_items','volume(litre)':'total_volume'},inplace=True)
by_brand=by_brand.merge(temp3,on=['region','category','sub_category'],how='left')
by_brand['value(%)']=by_brand['value(kzt)']/by_brand['total_value']*100
by_brand['items(%)']=by_brand['items']/by_brand['total_items']*100
by_brand['volume(%)']=by_brand['volume(litre)']/by_brand['total_volume']*100
by_brand.drop(['total_value','total_items','total_volume'],axis=1,inplace=True)
by_brand.sort_values(['region', 'producer','brand'], inplace=True)


by_company=collect_cities_brand(by_company,barcode_information_table,level='producer')
temp4=by_company.groupby(by=['region','category','sub_category']).sum()[['value(kzt)','items','volume(litre)']].reset_index()
temp4.rename(columns={'value(kzt)':'total_value','items':'total_items','volume(litre)':'total_volume'},inplace=True)
by_company=by_company.merge(temp4,on=['region','category','sub_category'],how='left')
by_company['value(%)']=by_company['value(kzt)']/by_company['total_value']*100
by_company['items(%)']=by_company['items']/by_company['total_items']*100
by_company['volume(%)']=by_company['volume(litre)']/by_company['total_volume']*100
by_company.drop(['total_value','total_items','total_volume'],axis=1,inplace=True)
by_company.sort_values(['region', 'producer'], inplace=True)



with pd.ExcelWriter(f"C:\\Users\\svnduw\\Desktop\\Bekbol\\Chingis\\PRK\\result\\2021-12.xlsx") as writer:
    by_barcode.to_excel(writer, sheet_name="by_barcode",index=False)  
    by_brand.to_excel(writer, sheet_name="by_brand",index=False)
    by_company.to_excel(writer, sheet_name="by_company",index=False) 



In [ ]:
by_brand.isnull().sum()

In [ ]:
by_barcode.isnull().sum()

In [ ]:
by_company.isnull().sum()

# Clear outliers

In [ ]:
barcode_information_table=pd.read_excel(r'C:\Users\svnduw\Desktop\Bekbol\Chingis\PRK\barcode_information.xlsx')
barcode_information_table.barcode=barcode_information_table.barcode.astype(np.int64)
# d_month = {'january':'2021-01', 'february':'2021-02','march':'2021-03', 'april':'2021-04',
#           'may':'2021-05', 'june':'2021-06','july':'2021-07','august':'2021-08',
#           'september':'2021-09','october':'2021-10','november':'2021-11','december':'2021-12'} 
d_month = {'december':'2021-12'} 
for month in list(d_month.keys()):
    main_data = pd.read_csv(r'D:\Bekbol working\by month\\'+d_month[month]+'.csv')
    stores_data=pd.read_csv(r'C:\Users\svnduw\Desktop\Bekbol\Chingis\general_data\store_id.csv')
    stores_data=stores_data[(stores_data[f'{str(month)}']==True) & (stores_data['region']!='Bishkek')]
#     main_data['barcode']=main_data['barcode'].astype(str)
    df=main_data[main_data['store_id'].isin(stores_data.store_id.tolist())]
    df=df[df['barcode'].isin(barcode_information_table.barcode.tolist())]
    df['price']=df['total_sale_amount']/df['product_quantity']
    df['region']=df['store_id'].map(dict(zip(stores_data['store_id'],stores_data['region'])))
    df=df[df['product_quantity']!=0]
    df=df[df['total_sale_amount']!=0]
#     b=df
# #     df.dropna(subset=['region'],inplace=True)
# #     temp=df.groupby(by=['barcode']).agg({'price':['mean', 'min', 'max']}).reset_index(level=[0])
#     for index,row in temp.iterrows():    
#             if row['price']['min']>=row['price']['max']*0.3:        
#                 temp.at[index,'check']=True
#             else:
#                 temp.at[index,'check']=False
#     a=temp[temp.check==False]
#     print(len(a.barcode.unique()))
#     for index2,row2 in a.iterrows():
#         a.at[index2,'check']=row2['price']['max']*0.5
#     for index3,row3 in a.iterrows():
#         temp3=df[df['barcode'].isin([row3['barcode'][0]])]
#         df=df[~df['barcode'].isin([row3['barcode'][0]])]
#         for index4,row4 in temp3.iterrows():
#             if row4['price']<row3['check'][0]:
#                 temp3.drop(index4,inplace=True)
#             else:
#                 continue
#         df.append(temp3,ignore_index=True)
#     df.to_csv(r'D:\Bekbol working\PRK\\'+d_month[month]+'.csv',index=False)
    print(month)
    

In [ ]:
temp=df.groupby(by=['region','barcode']).agg({'price':['mean', 'min', 'max',lambda x: pd.Series.mode(x).iat[0]]}).reset_index()
for index,row in temp.iterrows():    
    if row['price']['min']>row['price']['max']*0.5:        
        temp.at[index,'check']=True
    else:
        temp.at[index,'check']=False
a=temp[temp.check==False]
# for index3,row3 in a.iterrows():
#     a.at[index3,'check2']=row3['price']['<lambda_0>']*0.05 
# for index4,row4 in a.iterrows():
#     temp4=df[df['barcode'].isin([row4['barcode'][0]])]
#     df=df[~df['barcode'].isin([row4['barcode'][0]])]
#     for index5,row5 in temp4.iterrows():
#         if (row5['region']==row4['region'][0]) and (row5['price']<row4['check2'][0]):
#             temp4.drop(index5,inplace=True)
#             df.append(temp4,ignore_index=True)

In [ ]:
a[100:]

In [ ]:
df[(df.barcode==5010852025309) & (df.region=='Shymkent')]
# df[(df.barcode==4870022002988) & (df.region=='Almaty - 21') & (df.price==8.0)]

In [ ]:
df.at[3451199,'price']=2000.0
df['total_sale_amount']=df['price']*df['product_quantity']

In [ ]:
print(len(df))        

In [ ]:
df.to_csv(r'D:\Bekbol working\PRK\2021-12.csv',index=False)

In [ ]:
d_month.values()

# Collecting and adding 

In [ ]:
# folders=['assorti','biscuit','waffers','bar']
folders=['bar']
result_barcode=pd.DataFrame()
result_brand=pd.DataFrame()
result_company=pd.DataFrame()

for folder in folders:
    path=f'C:\\Users\\svnduw\\Desktop\\Bekbol\\Chingis\\rakhat\\result\\report\\{folder}'
    files=os.listdir(path)
    for file in files:
        if file.endswith('xlsx'):            
            name=str(file.split('_')[1].split('.')[0].strip())+' 2021'
            barcode_information=pd.read_excel(r'C:\Users\svnduw\Desktop\Bekbol\Chingis\rakhat\result\output.xlsx',sheet_name=str(folder))
            barcode_information.barcode=barcode_information.barcode.astype(str)
            df_barcode=pd.read_excel(f'{path}//{file}',sheet_name='by_barcode')
            df_barcode.barcode=df_barcode.barcode.astype(str)
            df_brand=pd.read_excel(f'{path}//{file}',sheet_name='by_brand')
            df_company=pd.read_excel(f'{path}//{file}',sheet_name='by_company')
            df_barcode['type']=df_barcode['barcode'].map(dict(zip(barcode_information['barcode'],barcode_information['type'])))
            df_barcode['ingredient']=df_barcode['barcode'].map(dict(zip(barcode_information['barcode'],barcode_information['ingredient'])))
#             df_barcode['flavour']=df_barcode['barcode'].map(dict(zip(barcode_information['barcode'],barcode_information['flavour'])))
            df_barcode['brand_origin']=df_barcode['barcode'].map(dict(zip(barcode_information['barcode'],barcode_information['brand_origin'])))
            df_barcode['month']=str(name)            
            df_brand['sub_subcategory']=str(df_barcode['sub_subcategory'].unique()[0])
            df_brand['month']=str(name)
            df_company['sub_subcategory']=str(df_barcode['sub_subcategory'].unique()[0])
            df_company['month']=str(name)
            result_barcode=result_barcode.append(df_barcode,ignore_index=True)
            result_brand=result_brand.append(df_brand,ignore_index=True)
            result_company=result_company.append(df_company,ignore_index=True)
            
            
#             with pd.ExcelWriter(f'{path}\\add\\{name}.xlsx') as writer:  
#                 df_barcode.to_excel(writer, sheet_name='by_barcode',index=False)
#                 df_brand.to_excel(writer, sheet_name='by_brand',index=False)
#                 df_company.to_excel(writer, sheet_name='by_company',index=False)
    print(folder,' done')
    
    

In [ ]:
all_barcode=pd.read_excel(r'C:\Users\svnduw\Desktop\Bekbol\Chingis\rakhat\result\report\all.xlsx',sheet_name='by_barcode')
all_brand=pd.read_excel(r'C:\Users\svnduw\Desktop\Bekbol\Chingis\rakhat\result\report\all.xlsx',sheet_name='by_brand')
all_company=pd.read_excel(r'C:\Users\svnduw\Desktop\Bekbol\Chingis\rakhat\result\report\all.xlsx',sheet_name='by_company')

In [ ]:
all_barcode=all_barcode.append(result_barcode,ignore_index=True)
all_brand=all_brand.append(result_brand,ignore_index=True)
all_company=all_company.append(result_company,ignore_index=True)

In [ ]:
with pd.ExcelWriter(r'C:\Users\svnduw\Desktop\Bekbol\Chingis\rakhat\result\report\all.xlsx') as writer:  
    all_barcode.to_excel(writer, sheet_name='by_barcode',index=False)
    all_brand.to_excel(writer, sheet_name='by_brand',index=False)
    all_company.to_excel(writer, sheet_name='by_company',index=False)

In [ ]:
path=r'C:\Users\svnduw\Desktop\Bekbol\Chingis\rakhat\result\report\bar'
files=os.listdir(path)
for file in files:
    if file.endswith('xlsx'):
        name=str(file.split('_')[1].split('.')[0].strip())+' 2021'
        barcode_information=pd.read_excel(r'C:\Users\svnduw\Desktop\Bekbol\Chingis\rakhat\result\output.xlsx',sheet_name='bar')
        barcode_information.barcode=barcode_information.barcode.astype(str)
        df_barcode=pd.read_excel(f'{path}//{file}',sheet_name='by_barcode')
        df_barcode.barcode=df_barcode.barcode.astype(str)
        df_brand=pd.read_excel(f'{path}//{file}',sheet_name='by_brand')
        df_company=pd.read_excel(f'{path}//{file}',sheet_name='by_company')
        df_barcode['type']=df_barcode['barcode'].map(dict(zip(barcode_information['barcode'],barcode_information['type'])))
        df_barcode['ingredient']=df_barcode['barcode'].map(dict(zip(barcode_information['barcode'],barcode_information['ingredient'])))
        df_barcode['brand_origin']=df_barcode['barcode'].map(dict(zip(barcode_information['barcode'],barcode_information['brand_origin'])))
        df_barcode['month']=str(name)
        df_brand['month']=str(name)
        df_company['month']=str(name)
        with pd.ExcelWriter(f'{path}\\add\\{name}.xlsx') as writer:  
            df_barcode.to_excel(writer, sheet_name='by_barcode',index=False)
            df_brand.to_excel(writer, sheet_name='by_brand',index=False)
            df_company.to_excel(writer, sheet_name='by_company',index=False)
                                                                

In [ ]:
import pandas as pd
import os
file_names=['april 2021.xlsx',
'august 2021.xlsx',
'december 2021.xlsx',
'february 2021.xlsx',
'january 2021.xlsx',
'july 2021.xlsx',
'june 2021.xlsx',
'march 2021.xlsx',
'may 2021.xlsx',
'november 2021.xlsx',
'october 2021.xlsx',
'september 2021.xlsx']
path_assorti=r'C:\Users\svnduw\Desktop\Bekbol\Chingis\rakhat\result\report\all_report\assorti_biscuit_waffers'
path_bar=r'C:\Users\svnduw\Desktop\Bekbol\Chingis\rakhat\result\report\all_report\bar'
path_biscuit=r'C:\Users\svnduw\Desktop\Bekbol\Chingis\rakhat\result\report\all_report\biscuit'
path_waff=r'C:\Users\svnduw\Desktop\Bekbol\Chingis\rakhat\result\report\all_report\waffers'
for file in file_names[:1]:
    files_assori=os.listdir(path_assorti)
    files_bar=os.listdir(path_bar)
    files_biscuit=os.listdir(path_biscuit)
    files_waff=os.listdir(path_waff)
    for file_assorti in files_assori:
        if str(file)==str(file_assorti):
            assorti_barcode=pd.read_excel(f'{path_assorti}//{file_assorti}',sheet_name='by_barcode') 
            assorti_brand=pd.read_excel(f'{path_assorti}//{file_assorti}',sheet_name='by_brand')
            assorti_company=pd.read_excel(f'{path_assorti}//{file_assorti}',sheet_name='by_company') 
    for file_bar in files_bar:
        if str(file)==str(file_bar):
            bar_barcode=pd.read_excel(f'{path_bar}//{file_bar}',sheet_name='by_barcode') 
            bar_brand=pd.read_excel(f'{path_bar}//{file_bar}',sheet_name='by_brand')
            bar_company=pd.read_excel(f'{path_bar}//{file_bar}',sheet_name='by_company')
    for file_biscuit in files_biscuit:
        if str(file)==str(file_biscuit):
            biscuit_barcode=pd.read_excel(f'{path_biscuit}//{file_biscuit}',sheet_name='by_barcode') 
            biscuit_brand=pd.read_excel(f'{path_biscuit}//{file_biscuit}',sheet_name='by_brand')
            biscuit_company=pd.read_excel(f'{path_biscuit}//{file_biscuit}',sheet_name='by_company')
    for file_waff in files_waff:
        if str(file)==str(file_waff):
            waff_barcode=pd.read_excel(f'{path_waff}//{file_waff}',sheet_name='by_barcode') 
            waff_brand=pd.read_excel(f'{path_waff}//{file_waff}',sheet_name='by_brand')
            waff_company=pd.read_excel(f'{path_waff}//{file_waff}',sheet_name='by_company')


In [ ]:
import os
import pandas as pd
import numpy as np
path=r'C:\Users\svnduw\Desktop\Bekbol\Chingis\PRK\result'
barcode_information=pd.read_excel(r'C:\Users\svnduw\Desktop\Bekbol\Chingis\PRK\barcode_information.xlsx')
barcode_information.barcode=barcode_information.barcode.astype(np.int64)
for file in files:
    name=file.split('.')[0]
    by_barcode=pd.read_excel(f'{path}//{file}',sheet_name='by_barcode')
    by_brand=pd.read_excel(f'{path}//{file}',sheet_name='by_brand')
    by_company=pd.read_excel(f'{path}//{file}',sheet_name='by_company')
    by_barcode['type']=by_barcode['barcode'].map(dict(zip(barcode_information['barcode'],barcode_information['sub_subcategory'])))
    with pd.ExcelWriter(f'{path}\\add\\{name}.xlsx') as writer:  
            by_barcode.to_excel(writer, sheet_name='by_barcode',index=False)
            by_brand.to_excel(writer, sheet_name='by_brand',index=False)
            by_company.to_excel(writer, sheet_name='by_company',index=False)
    